In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#unzipping train dataset
!mkdir data
!unzip /content/drive/MyDrive/data291.zip -d /content/data/
!mkdir data/datalr

In [ ]:
#unzipping test dataset
!unzip /content/drive/MyDrive/set5.zip -d /content/data/

In [ ]:
!unzip /content/drive/MyDrive/set14.zip -d /content/data

In [9]:
!mkdir data/model
!mkdir data/output

In [6]:
import PIL.Image as pil_image

#resize set5


image_link = "/content/data/set5"


for j in range(2,5):
  scale = j
  for i in range (1,6):
      image_file = image_link +"/s" +str(i)+ ".png"
      image = pil_image.open(image_file)
      image_width = image.width
      image_height = image.height
      image = image.resize((image.width//scale, image.height//scale))
      image = image.resize((image_width, image_height), resample=pil_image.BICUBIC)
      image.save( image_link +"/s"+ str(i)+"lr" + str(scale) + ".png")

In [7]:
import PIL.Image as pil_image

#resize set14


image_link = "/content/data/set14"


for j in range(2,5):
  scale = j
  for i in range (1,15):
      image_file = image_link +"/ss" +str(i)+ ".png"
      image = pil_image.open(image_file)
      image_width = image.width
      image_height = image.height
      image = image.resize((image.width//scale, image.height//scale))
      image = image.resize((image_width, image_height), resample=pil_image.BICUBIC)
      image.save( image_link +"/ss"+ str(i)+"lr" + str(scale) + ".png")

In [8]:
import PIL.Image as pil_image

#resize image to create Low Resolution image


image_link = "/content/data/data"

for j in range(2,5):
  scale = j
  for i in range (1,292):
      image_file = image_link +"/t" +str(i)+ ".png"
      image = pil_image.open(image_file)
      image_width = image.width
      image_height = image.height
      image = image.resize((image.width//scale, image.height//scale))
      image = image.resize((image_width, image_height), resample=pil_image.BICUBIC)
      image.save( image_link +"lr/t"+ str(i)+"lr" + str(scale) + ".png")

In [ ]:
import torch
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset
import torch.optim as optim
from torch.utils.data.dataloader import DataLoader
from PIL import Image
from numpy import asarray
import PIL.Image as pil_image
from torchvision import transforms
import random
import PIL


def psnr_between_rgb(img1,img2):
    if len(img1.shape) == 4:
        img1 = img1.squeeze(0)
        img2 = img2.squeeze(0)
    y1 = 16. + (64.738 * img1[0, :, :] + 129.057 * img1[1, :, :] + 25.064 * img1[2, :, :]) / 256.
    y2 = 16. + (64.738 * img2[0, :, :] + 129.057 * img2[1, :, :] + 25.064 * img2[2, :, :]) / 256.
    psnr = (10. * torch.log10(1. / torch.mean((y1 - y2) ** 2)))
    return psnr


### transform
transform_train = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()

])

####################################################################################
#########  MODE SELECTION

###    Train with new model    = 1
### Train with existing model  = 2
###      Test with set5        = 3
###     Test with set14        = 4

mode = 3

transform = True


##########  MODEL
model_parameters_path = "/content/data/model/model.pth"


######### PARAMETER
learning_rate = 0.01
training_epochs = 20
batch_size = 16
userseed = 123

######### FILE ROUTES


######### MODEL PARAMETER
channel_1 = 64
channel_2 = 64
channel_3 = 64

kernel_1 = 3
kernel_2 = 3  
kernel_3 = 3

####### GRADIENT CLIPPING
max_norm = 0.0000000005

####### test parameter
test_scale = 3


#####################################################################################






lrarr2 = []
lrarr3 = []
lrarr4 = []
hrarr = []
image_link = "/content/data/data"
sub_image_size = 41
stride = 41
for i in range (1,292): # total number of image + 1


    ### image file opening
    lr2image = pil_image.open(image_link + "lr/t" + str(i) + "lr2.png")
    lr3image = pil_image.open(image_link + "lr/t" + str(i) + "lr3.png")
    lr4image = pil_image.open(image_link + "lr/t" + str(i) + "lr4.png")
    hrimage = pil_image.open(image_link + "/t" + str(i) + ".png")


    for j in range(0, ( lr2image.size[0]-sub_image_size)//stride +1):
        for k in range(0 , (lr2image.size[1]-sub_image_size)//stride +1):
                
            ### crop image
            lr2cropped = lr2image.crop((j*stride, k*stride, j*stride+sub_image_size, k*stride+sub_image_size))
            lr3cropped = lr3image.crop((j*stride, k*stride, j*stride+sub_image_size, k*stride+sub_image_size))
            lr4cropped = lr4image.crop((j*stride, k*stride, j*stride+sub_image_size, k*stride+sub_image_size)) 
            hrcropped = hrimage.crop((j*stride, k*stride, j*stride+sub_image_size, k*stride+sub_image_size))

            ### post processing of image file
            lr2croppedimage = np.array(lr2cropped)
            lr3croppedimage = np.array(lr3cropped)
            lr4croppedimage = np.array(lr4cropped)  
            hrcroppedimage = np.array(hrcropped)    
                
            lrarr2.append(lr2croppedimage)
            lrarr3.append(lr3croppedimage)
            lrarr4.append(lr4croppedimage)
            hrarr.append(hrcroppedimage)

    for j in range(0, ( lr2image.size[0]-sub_image_size)//stride +1):
      lr2cropped = lr2image.crop((j*stride, lr2image.size[1]-sub_image_size, j*stride+sub_image_size, lr2image.size[1]))
      lr3cropped = lr3image.crop((j*stride, lr2image.size[1]-sub_image_size, j*stride+sub_image_size, lr2image.size[1]))
      lr4cropped = lr4image.crop((j*stride, lr2image.size[1]-sub_image_size, j*stride+sub_image_size, lr2image.size[1]))
      hrcropped = hrimage.crop((j*stride, lr2image.size[1]-sub_image_size, j*stride+sub_image_size, lr2image.size[1]))

      lr2croppedimage = np.array(lr2cropped)
      lr3croppedimage = np.array(lr3cropped)
      lr4croppedimage = np.array(lr4cropped)
      hrcroppedimage = np.array(hrcropped) 

      lrarr2.append(lr2croppedimage)
      lrarr3.append(lr3croppedimage)
      lrarr4.append(lr4croppedimage)
      hrarr.append(hrcroppedimage)
 

    for k in range(0 , (lr2image.size[1]-sub_image_size)//stride +1):
      lr2cropped = lr2image.crop((lr2image.size[0]-sub_image_size, k*stride, lr2image.size[0], k*stride+sub_image_size))
      lr3cropped = lr3image.crop((lr2image.size[0]-sub_image_size, k*stride, lr2image.size[0], k*stride+sub_image_size))
      lr4cropped = lr4image.crop((lr2image.size[0]-sub_image_size, k*stride, lr2image.size[0], k*stride+sub_image_size))
      hrcropped = hrimage.crop((lr2image.size[0]-sub_image_size, k*stride, lr2image.size[0], k*stride+sub_image_size))
      
      lr2croppedimage = np.array(lr2cropped)
      lr3croppedimage = np.array(lr3cropped)
      lr4croppedimage = np.array(lr4cropped)
      hrcroppedimage = np.array(hrcropped) 

      lrarr2.append(lr2croppedimage)
      lrarr3.append(lr3croppedimage)
      lrarr4.append(lr4croppedimage)
      hrarr.append(hrcroppedimage)
    
    lr2cropped = lr2image.crop((lr2image.size[0]-sub_image_size, lr2image.size[1]-sub_image_size, lr2image.size[0], lr2image.size[1]))
    lr3cropped = lr3image.crop((lr2image.size[0]-sub_image_size, lr2image.size[1]-sub_image_size, lr2image.size[0], lr2image.size[1]))
    lr4cropped = lr4image.crop((lr2image.size[0]-sub_image_size, lr2image.size[1]-sub_image_size, lr2image.size[0], lr2image.size[1]))
    hrcropped = hrimage.crop((lr2image.size[0]-sub_image_size, lr2image.size[1]-sub_image_size, lr2image.size[0], lr2image.size[1]))

    lr2croppedimage = np.array(lr2cropped)
    lr3croppedimage = np.array(lr3cropped)
    lr4croppedimage = np.array(lr4cropped)
    hrcroppedimage = np.array(hrcropped) 

    lrarr2.append(lr2croppedimage)
    lrarr3.append(lr3croppedimage)
    lrarr4.append(lr4croppedimage)
    hrarr.append(hrcroppedimage)


class VDSR(nn.Module):
    def __init__(self):
        super().__init__()


        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, channel_1, kernel_size=kernel_1, stride=1, padding = kernel_1//2),
            torch.nn.ReLU(), 
        )
        torch.nn.init.kaiming_uniform_(self.layer1[0].weight)
        

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(channel_1, channel_2, kernel_size=kernel_2, stride=1, padding = kernel_2//2),
            torch.nn.ReLU(), 
        )
        torch.nn.init.kaiming_uniform_(self.layer2[0].weight)

        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(channel_1, channel_2, kernel_size=kernel_2, stride=1, padding = kernel_2//2),
            torch.nn.ReLU(), 
        )
        torch.nn.init.kaiming_uniform_(self.layer3[0].weight)

        self.layer4 = torch.nn.Sequential(
            torch.nn.Conv2d(channel_1, channel_2, kernel_size=kernel_2, stride=1, padding = kernel_2//2),
            torch.nn.ReLU(), 
        )
        torch.nn.init.kaiming_uniform_(self.layer4[0].weight)

        self.layer5 = torch.nn.Sequential(
            torch.nn.Conv2d(channel_1, channel_2, kernel_size=kernel_2, stride=1, padding = kernel_2//2),
            torch.nn.ReLU(), 
        )
        torch.nn.init.kaiming_uniform_(self.layer5[0].weight)

        self.layer6 = torch.nn.Sequential(
            torch.nn.Conv2d(channel_1, channel_2, kernel_size=kernel_2, stride=1, padding = kernel_2//2),
            torch.nn.ReLU(), 
        )
        torch.nn.init.kaiming_uniform_(self.layer6[0].weight)

        self.layer7 = torch.nn.Sequential(
            torch.nn.Conv2d(channel_1, channel_2, kernel_size=kernel_2, stride=1, padding = kernel_2//2),
            torch.nn.ReLU(), 
        )
        torch.nn.init.kaiming_uniform_(self.layer7[0].weight)

        self.layer8 = torch.nn.Sequential(
            torch.nn.Conv2d(channel_1, channel_2, kernel_size=kernel_2, stride=1, padding = kernel_2//2),
            torch.nn.ReLU(), 
        )
        torch.nn.init.kaiming_uniform_(self.layer8[0].weight)
        
        self.layer9 = torch.nn.Sequential(
            torch.nn.Conv2d(channel_1, channel_2, kernel_size=kernel_2, stride=1, padding = kernel_2//2),
            torch.nn.ReLU(), 
        )
        torch.nn.init.kaiming_uniform_(self.layer9[0].weight)

        self.layer10 = torch.nn.Sequential(
            torch.nn.Conv2d(channel_1, channel_2, kernel_size=kernel_2, stride=1, padding = kernel_2//2),
            torch.nn.ReLU(), 
        )
        torch.nn.init.kaiming_uniform_(self.layer10[0].weight)

        self.layer11 = torch.nn.Sequential(
            torch.nn.Conv2d(channel_1, channel_2, kernel_size=kernel_2, stride=1, padding = kernel_2//2),
            torch.nn.ReLU(), 
        )
        torch.nn.init.kaiming_uniform_(self.layer11[0].weight)

        self.layer12 = torch.nn.Sequential(
            torch.nn.Conv2d(channel_1, channel_2, kernel_size=kernel_2, stride=1, padding = kernel_2//2),
            torch.nn.ReLU(), 
        )
        torch.nn.init.kaiming_uniform_(self.layer12[0].weight)

        self.layer13 = torch.nn.Sequential(
            torch.nn.Conv2d(channel_1, channel_2, kernel_size=kernel_2, stride=1, padding = kernel_2//2),
            torch.nn.ReLU(), 
        )
        torch.nn.init.kaiming_uniform_(self.layer13[0].weight)

        self.layer14 = torch.nn.Sequential(
            torch.nn.Conv2d(channel_1, channel_2, kernel_size=kernel_2, stride=1, padding = kernel_2//2),
            torch.nn.ReLU(), 
        )
        torch.nn.init.kaiming_uniform_(self.layer14[0].weight)

        self.layer15 = torch.nn.Sequential(
            torch.nn.Conv2d(channel_1, channel_2, kernel_size=kernel_2, stride=1, padding = kernel_2//2),
            torch.nn.ReLU(), 
        )
        torch.nn.init.kaiming_uniform_(self.layer15[0].weight)

        self.layer16 = torch.nn.Sequential(
            torch.nn.Conv2d(channel_1, channel_2, kernel_size=kernel_2, stride=1, padding = kernel_2//2),
            torch.nn.ReLU(), 
        )
        torch.nn.init.kaiming_uniform_(self.layer16[0].weight)

        self.layer17 = torch.nn.Sequential(
            torch.nn.Conv2d(channel_1, channel_2, kernel_size=kernel_2, stride=1, padding = kernel_2//2),
            torch.nn.ReLU(), 
        )
        torch.nn.init.kaiming_uniform_(self.layer17[0].weight)

        self.layer18 = torch.nn.Sequential(
            torch.nn.Conv2d(channel_1, channel_2, kernel_size=kernel_2, stride=1, padding = kernel_2//2),
            torch.nn.ReLU(), 
        )
        torch.nn.init.kaiming_uniform_(self.layer18[0].weight)

        self.layer19 = torch.nn.Sequential(
            torch.nn.Conv2d(channel_1, channel_2, kernel_size=kernel_2, stride=1, padding = kernel_2//2),
            torch.nn.ReLU(), 
        )
        torch.nn.init.kaiming_uniform_(self.layer19[0].weight)
      

        self.layer20 = torch.nn.Sequential(
            torch.nn.Conv2d(channel_2, 3, kernel_size=kernel_3, stride=1, padding = kernel_3//2),
        )
        torch.nn.init.kaiming_uniform_(self.layer20[0].weight)
        

    def forward(self,x):
        residual = x
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        out = self.layer14(out)
        out = self.layer15(out)
        out = self.layer16(out)
        out = self.layer17(out)
        out = self.layer18(out)
        out = self.layer19(out)
        out = self.layer20(out)
        out = torch.add(out,residual)
        return out

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(userseed)

if device == 'cuda':
    torch.cuda.manual_seed_all(userseed)


### model load

model = VDSR().to(device)

if mode ==2 or mode ==3 or mode==4:
    model.load_state_dict(torch.load(model_parameters_path))


criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### dataset
class TrainDataset(Dataset):
    def __init__(self, transform = None):
        super(TrainDataset,self).__init__()
        self.transform = transform
        self.transform_train_1 = transforms.Compose([ # VERTICAL
        transforms.ToPILImage(),
        transforms.RandomVerticalFlip(p = 1),
        transforms.ToTensor()
        ])
        self.transform_train_2 = transforms.Compose([ # HORIZONTAL
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(p = 1),
        transforms.ToTensor()
        ])
        self.transform_train_3 = transforms.Compose([# VERTICAL + HORIZONTAL
        transforms.ToPILImage(),
        transforms.RandomVerticalFlip(p = 1),
        transforms.RandomHorizontalFlip(p = 1),
        transforms.ToTensor()
        ])
        self.transform_train_4 = transforms.Compose([ # 90 ROTATION
        transforms.ToPILImage(),
        transforms.RandomRotation(degrees = (90,90)),
        transforms.ToTensor()
        ])
        self.transform_train_5 = transforms.Compose([ #270 ROTATION
        transforms.ToPILImage(),
        transforms.RandomRotation(degrees = (270,270)),
        transforms.ToTensor()
        ])
        self.transform_train_6 = transforms.Compose([ #VERTICAL + 90
        transforms.ToPILImage(),
        transforms.RandomVerticalFlip(p = 1),
        transforms.RandomRotation(degrees = (90,90)),
        transforms.ToTensor()
        ])
        self.transform_train_7 = transforms.Compose([ #HORIZONTAL + 90
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(p = 1),
        transforms.RandomRotation(degrees = (90,90)),
        transforms.ToTensor()
        ])

    def __getitem__(self,idx):

        if self.transform:
            # y = random parameter for scale
            y = random.random()
            if(y<0.334):
              lr = lrarr2[idx]
            elif(y<0.667):
              lr = lrarr3[idx]
            else:
              lr = lrarr4[idx]
            hr = hrarr[idx]
            
            # x = random parameter for rotation / flip
            x = random.random()
            
            if(x<0.125):
                return self.transform_train_1(lr).to("cpu").numpy().astype(np.float32) ,self.transform_train_1(hrarr[idx]).to("cpu").numpy().astype(np.float32)
            elif(x<0.25):
                return self.transform_train_2(lr).to("cpu").numpy().astype(np.float32) ,self.transform_train_2(hrarr[idx]).to("cpu").numpy().astype(np.float32)
            elif(x<0.375):
                return self.transform_train_3(lr).to("cpu").numpy().astype(np.float32) ,self.transform_train_3(hrarr[idx]).to("cpu").numpy().astype(np.float32)
            elif(x<0.5):
                return self.transform_train_4(lr).to("cpu").numpy().astype(np.float32) ,self.transform_train_4(hrarr[idx]).to("cpu").numpy().astype(np.float32)
            elif(x<0.625):
                return self.transform_train_5(lr).to("cpu").numpy().astype(np.float32) ,self.transform_train_5(hrarr[idx]).to("cpu").numpy().astype(np.float32)
            elif(x<0.75):
                return self.transform_train_6(lr).to("cpu").numpy().astype(np.float32) ,self.transform_train_6(hrarr[idx]).to("cpu").numpy().astype(np.float32)
            elif(x<0.875):
                return self.transform_train_7(lr).to("cpu").numpy().astype(np.float32) ,self.transform_train_7(hrarr[idx]).to("cpu").numpy().astype(np.float32)
            else:
                return lr.transpose((2,0,1)).astype(np.float32)/255. ,hrarr[idx].transpose((2,0,1)).astype(np.float32)/255.
            
        return lr.transpose((2,0,1)).astype(np.float32)/255. ,hrarr[idx].transpose((2,0,1)).astype(np.float32)/255. 
        
        
    def __len__(self):
        return len(hrarr)

    

if transform:
    train_dataset = TrainDataset(transform = transform_train)
else:
    train_dataset = TrainDataset()

### dataloader
train_dataloader = DataLoader(dataset = train_dataset,
                 batch_size = 16,
                 shuffle = True,
                 num_workers = 2,
                 pin_memory=True,
                 drop_last = True)

if mode ==1 or mode==2:

  for epoch in range(training_epochs):
        model.train()
        avg_cost=0
        sum_psnr1 = 0
        count = 0
        for data in train_dataloader:
            count = count+1
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            
            preds = model(inputs)
            
            
            loss = criterion(preds, labels) #MSE
            
            psnr1 = psnr_between_rgb(inputs,labels)
            sum_psnr1 = sum_psnr1+psnr1






            optimizer.zero_grad()

            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm/learning_rate)
            optimizer.step()
            avg_cost += loss / batch_size


        print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
        model.eval()
        sum_psnr = 0
        

        for i in range(1,6):

          setlrlr = Image.open("/content/data/set5/s" + str(i) + "lr" + str(test_scale)+".png")
          setlrlr = np.array(asarray(setlrlr)).astype(np.float32)
          setlrlr = setlrlr.transpose((2,0,1))
          setlrlr /= 255.
          setlrlr = torch.from_numpy(setlrlr).to(device)
          

          sethrhr = Image.open("/content/data/set5/s"+str(i)+".png")
          sethrhr= np.array(asarray(sethrhr)).astype(np.float32)
          sethrhr = sethrhr.transpose((2,0,1))
          sethrhr /= 255.
          sethrhr = torch.from_numpy(sethrhr).to(device)


          with torch.no_grad():
              setpreds = model(setlrlr).clamp(0.0,1.0)

          psnr = psnr_between_rgb(sethrhr,setpreds)
          sum_psnr+=psnr
          
        print('PSNR: {:.2f}'.format(sum_psnr/5))

  torch.save(model.state_dict(), model_parameters_path)

### only testing - set5
elif mode == 3:
  model.eval()
  sum_psnr = 0
 
  for i in range(1,6):
    setlrlr = Image.open("/content/data/set5/s" + str(i) + "lr" + str(test_scale)+".png")
    setlrlr = np.array(asarray(setlrlr)).astype(np.float32)
    setlrlr = setlrlr.transpose((2,0,1))
    setlrlr /= 255.
    setlrlr = torch.from_numpy(setlrlr).to(device)
          

    sethrhr = Image.open("/content/data/set5/s"+str(i)+".png")
    sethrhr= np.array(asarray(sethrhr)).astype(np.float32)
    sethrhr = sethrhr.transpose((2,0,1))
    sethrhr /= 255.
    sethrhr = torch.from_numpy(sethrhr).to(device)

    

    with torch.no_grad():
        setpreds = model(setlrlr).clamp(0.0,1.0)
    
    
    #save part 
    '''
    preds2 = setpreds.mul(255.0).cpu().numpy()

    preds2 = preds2.squeeze()
    preds2 = preds2.transpose((1,2,0))
    preds2 = np.clip(preds2, 0.0,255.0).astype(np.uint8)
    output = pil_image.fromarray(preds2)
    output.save("/content/data/output/output"+str(i)+".png")
    '''
    psnr = psnr_between_rgb(sethrhr,setpreds)
    sum_psnr+=psnr
          
  print('PSNR: {:.2f}'.format(sum_psnr/5))

### testing set14 
elif mode ==4:
  model.eval()
  sum_psnr = 0
   
    

  for i in range(1,15):
    setlrlr = Image.open( "/content/data/set14/ss" + str(i) + "lr" + str(test_scale)+".png")
    setlrlr = np.array(asarray(setlrlr)).astype(np.float32)
    setlrlr = setlrlr.transpose((2,0,1))
    setlrlr /= 255.
    setlrlr = torch.from_numpy(setlrlr).to(device)
          

    sethrhr = Image.open( "/content/data/set14/ss" + str(i) + ".png")
    sethrhr= np.array(asarray(sethrhr)).astype(np.float32)
    sethrhr = sethrhr.transpose((2,0,1))
    sethrhr /= 255.
    sethrhr = torch.from_numpy(sethrhr).to(device)


    with torch.no_grad():
        setpreds = model(setlrlr).clamp(0.0,1.0)

    
    
    
    #save part 
    '''
    preds2 = setpreds.mul(255.0).cpu().numpy()

    preds2 = preds2.squeeze()
    preds2 = preds2.transpose((1,2,0))
    preds2 = np.clip(preds2, 0.0,255.0).astype(np.uint8)
    output = pil_image.fromarray(preds2)
    output.save("/content/data/output2/output"+str(i)+".png")
    '''
    psnr = psnr_between_rgb(sethrhr,setpreds)
    sum_psnr+=psnr
  print('PSNR: {:.2f}'.format(sum_psnr/14))
